- Use Pandas read_html() function to retrive the HTML tables as dataframes
- from the web page: https://www.genealogybranches.com/censuscosts.html
- Find out how many HTML tables Pandas retrieves from the web page. Find 
- out which dataframe contain the data and use it for further processing.


In [1]:
import pandas as pd

In [2]:
df_list = pd.read_html("https://www.genealogybranches.com/censuscosts.html")
print("This is the number of tables in the html")
len(df_list)

This is the number of tables in the html


2

- Display and explore the data (rows, columns, etc.) and determine the data 
- quality (bad rows, bad columns, bad elements, null values, etc.)


In [3]:
df1 = df_list[0]
print("The first table's 5 columns are displayed")
df1.head()

The first table's 5 columns are displayed


,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [4]:
df2 = df_list[1]
print("The second table is displayed")
print(df2)

The second table is displayed
                                                   0
0  The U.S. population reached 300,000,000 on 17 ...


## Document the data quality issues: 
The data type is object which is not easily manuever which needed to be converted to integers or floats


In [5]:
print("These are the number of columns and row")
df1.info()

These are the number of columns and row
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       24 non-null     object
 1   1       24 non-null     object
 2   2       24 non-null     object
 3   3       24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes


It is best to view the whole file to see what needs to be cleanse. 

In [6]:
df1

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents
6,1840,17069458,"$833,371",4.88 cents
7,1850,23191876,"$1,423,351",6.14 cents
8,1860,31443321,"$1,969,377",6.26 cents
9,1870,38558371,"$3,421,198",8.87 cents


- There is an asterick on the row 23 and dollar "$" signs that needs to be removed. 
- In the average cost per person column, there are cents and dollar values. 
- There should be uniformity of cents in the average cost per person. 
- The headings are confusing which needed to be dropped, replaced and labeled properly.

In [7]:

Heading = df1.iloc[0]
Heading

0                Census Year
1           Total Population
2                Census Cost
3    Average Cost Per Person
Name: 0, dtype: object

In [8]:
df1.columns = Heading
df1.head(5)

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


In [9]:
# inplace will be used, so this dataframe will consistently be the newly
# revised heading
df1.drop([0], inplace=True)
df1.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents
5,1830,12866020,"$378,545",2.94 cents


Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. 
- Test the function by using test strings (for example, "1989*", "20*10")
- Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.
- Repeat this process for column "Total Population", "Census Cost", "Average Cost Per Person". (define, test, and apply the function)


In [10]:
def cleanse_cost(x):
    new = x.replace("*","")
    if  x.find("4.5") != -1:          # if "4.5" in x:
        new = "45" + "0" * 8
    elif x.find("Billion") != -1:     # elif "Billion" in x: 
        new = "13" + "0" * 9
    elif x.find("cents") != -1:       #elif "cents" in x:
        x = x.replace("cents", "")    #remove cents
        x = float(x)                  #change to float
        new = round(x/100, 4)         #round the new answers to 4 digits
    elif x.find(".") != -1:           #elif "." in x
        x = x.replace("$", "")        #remove "$"
        xfloat = float(x)             #converts x to float which is the dollar and cents
        xint = int(xfloat)            #converts x to int which is the dollar   
        xdec = xfloat - xint          #float minus int to get the decimal; x in cents or decimal
        dolxdec = round(xdec/100, 4)  #the cents is converted to dollar; dollar in rounded in 4 decimal
        new = xint + dolxdec          #new is dollar plus the cents converted to dollar
    elif x.find("$") != -1:
        x1 = x.replace("$", "")
        new = x1.replace(",", "")
    return new


In [11]:
df1["Revised CCost"] = df1["Census Cost"].apply(cleanse_cost)
df1.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person,Revised CCost
1,1790,3929214,"$44,377",1.13 cents,44377
2,1800,5308483,"$66,109",1.24 cents,66109
3,1810,7239881,"$178,445",2.46 cents,178445
4,1820,9633822,"$208,526",2.16 cents,208526
5,1830,12866020,"$378,545",2.94 cents,378545


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Census Year              23 non-null     object
 1   Total Population         23 non-null     object
 2   Census Cost              23 non-null     object
 3   Average Cost Per Person  23 non-null     object
 4   Revised CCost            23 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [13]:
df1["Revised CCost"]= df1["Revised CCost"].astype(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Census Year              23 non-null     object
 1   Total Population         23 non-null     object
 2   Census Cost              23 non-null     object
 3   Average Cost Per Person  23 non-null     object
 4   Revised CCost            23 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 1.1+ KB


In [14]:
df1["Revised CYear"] = df1["Census Year"].apply(cleanse_cost)
df1.tail(5)

,Census Year,Total Population,Census Cost,Average Cost Per Person,Revised CCost,Revised CYear
19,1970,203302031,"$247,653,000",$1.22,247653000,1970
20,1980,226542199,"$1,078,488,000",$4.76,1078488000,1980
21,1990,248718301,"$2,492,830,000",$10.02,2492830000,1990
22,2000,281421906,$4.5 Billion,$15.99,4500000000,2000
23,2010*,308745538,$13 Billion,$42.11,13000000000,2010


In [15]:
df1["Revised CYear"]= df1["Revised CYear"].astype(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Census Year              23 non-null     object
 1   Total Population         23 non-null     object
 2   Census Cost              23 non-null     object
 3   Average Cost Per Person  23 non-null     object
 4   Revised CCost            23 non-null     int64 
 5   Revised CYear            23 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 1.3+ KB


In [16]:
df1["Revised TPop"] = df1["Total Population"].apply(cleanse_cost)
df1.head()

,Census Year,Total Population,Census Cost,Average Cost Per Person,Revised CCost,Revised CYear,Revised TPop
1,1790,3929214,"$44,377",1.13 cents,44377,1790,3929214
2,1800,5308483,"$66,109",1.24 cents,66109,1800,5308483
3,1810,7239881,"$178,445",2.46 cents,178445,1810,7239881
4,1820,9633822,"$208,526",2.16 cents,208526,1820,9633822
5,1830,12866020,"$378,545",2.94 cents,378545,1830,12866020


In [17]:
df1["Revised TPop"]= df1["Revised TPop"].astype(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Census Year              23 non-null     object
 1   Total Population         23 non-null     object
 2   Census Cost              23 non-null     object
 3   Average Cost Per Person  23 non-null     object
 4   Revised CCost            23 non-null     int64 
 5   Revised CYear            23 non-null     int64 
 6   Revised TPop             23 non-null     int64 
dtypes: int64(3), object(4)
memory usage: 1.4+ KB


In [18]:
df1["Revised ACost"] = df1["Average Cost Per Person"].apply(cleanse_cost)
df1

,Census Year,Total Population,Census Cost,Average Cost Per Person,Revised CCost,Revised CYear,Revised TPop,Revised ACost
1,1790,3929214,"$44,377",1.13 cents,44377,1790,3929214,0.0113
2,1800,5308483,"$66,109",1.24 cents,66109,1800,5308483,0.0124
3,1810,7239881,"$178,445",2.46 cents,178445,1810,7239881,0.0246
4,1820,9633822,"$208,526",2.16 cents,208526,1820,9633822,0.0216
5,1830,12866020,"$378,545",2.94 cents,378545,1830,12866020,0.0294
6,1840,17069458,"$833,371",4.88 cents,833371,1840,17069458,0.0488
7,1850,23191876,"$1,423,351",6.14 cents,1423351,1850,23191876,0.0614
8,1860,31443321,"$1,969,377",6.26 cents,1969377,1860,31443321,0.0626
9,1870,38558371,"$3,421,198",8.87 cents,3421198,1870,38558371,0.0887
10,1880,50155783,"$5,790,678",11.54 cents,5790678,1880,50155783,0.1154


In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Census Year              23 non-null     object 
 1   Total Population         23 non-null     object 
 2   Census Cost              23 non-null     object 
 3   Average Cost Per Person  23 non-null     object 
 4   Revised CCost            23 non-null     int64  
 5   Revised CYear            23 non-null     int64  
 6   Revised TPop             23 non-null     int64  
 7   Revised ACost            23 non-null     float64
dtypes: float64(1), int64(3), object(4)
memory usage: 1.6+ KB


In [20]:
print("The statistic information for the Revised Census Cost")
df1["Revised CCost"].describe()

The statistic information for the Revised Census Cost


count    2.300000e+01
mean     9.445587e+08
std      2.830037e+09
min      4.437700e+04
25%      1.128361e+06
50%      1.185400e+07
75%      1.096980e+08
max      1.300000e+10
Name: Revised CCost, dtype: float64

In [21]:
print("These are the old headings.")
Heading

These are the old headings.


0                Census Year
1           Total Population
2                Census Cost
3    Average Cost Per Person
Name: 0, dtype: object

# New Headings Based On Old Headings
- Revised CYear meant Revised Census Year
- Revised TPop meant Revised Total Population
- Revised CCost meant Revised Census Cost
- Revised ACost meant Revised Average Cost Per Person

In [22]:
cleansed_census_cost = df1[["Revised CYear", "Revised TPop", "Revised CCost", "Revised ACost"]]
cleansed_census_cost

,Revised CYear,Revised TPop,Revised CCost,Revised ACost
1,1790,3929214,44377,0.0113
2,1800,5308483,66109,0.0124
3,1810,7239881,178445,0.0246
4,1820,9633822,208526,0.0216
5,1830,12866020,378545,0.0294
6,1840,17069458,833371,0.0488
7,1850,23191876,1423351,0.0614
8,1860,31443321,1969377,0.0626
9,1870,38558371,3421198,0.0887
10,1880,50155783,5790678,0.1154


- After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.
- Use Pandas to read the saved cleansed file and explore to make sure it is clean.
- Upload both the Jupyter notebook and the cleanse data file to GitHub and submit the GitHub link in BlackBoard.


In [23]:
CCC = cleansed_census_cost.to_csv('census_cost_cleansed.csv', index = True)